In [1]:
import gm2
import trfp

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

Welcome to JupyROOT 6.12/04


In [2]:
tr_run = gm2.Trolley([3956], prefix='v9_20_00/FieldPlainRootOutput_')
tr_time, tr_phi, tr_freq = tr_run.getBasics()

# drop first 5 events, which are 0, and last event, which can some times be 0
tr_time = tr_time[5:-91,:]/1.0e9  # convert nsec to sec
tr_phi = tr_phi[5:-91,:]
tr_freq = tr_freq[5:-91,:]
for tr in range(17):
    tr_freq[:, tr] += trfp.PLUNGING_PROBE_CALIBRATIONS[tr]

Reading event 8780/8781 99.99%          
loop done: 100%                                    


In [18]:
pos = tr_phi[:,8]

freq = np.zeros(tr_freq.shape)

freq[:,8] = tr_freq[:,8]
for i in range(17):
    if i == 8: continue
    freq[:,i] = np.interp(pos, tr_phi[:,i], tr_freq[:,i], period=360)
    
theta = trfp.THETA_TR
moment = np.transpose(np.matmul(theta, np.transpose(freq)))

In [19]:
sort_ind = np.argsort(pos)

In [20]:
weights = ((np.roll(pos[sort_ind], -1) - np.roll(pos[sort_ind], 1))%360)/720

In [21]:
print np.arange(len(weights))[weights == 0.0]

[6333]


In [22]:
ms = np.zeros(17)
for m in range(17):
    ms[m] = np.sum(weights*moment[sort_ind,m])

In [23]:
print ms

[ 5.20102223e+04 -1.31314418e+00  9.48085695e+00  1.74735318e+01
 -7.22058370e+01  1.32578073e-01  1.74605364e+01  3.45419787e+00
 -8.56247474e+01  1.75565220e+02  2.36577208e+02 -6.84174640e+01
  0.00000000e+00 -3.71530151e+02  4.11279356e+02  0.00000000e+00
  0.00000000e+00]


In [ ]:
plt.plot(pos, moment[:,0], '.')

In [17]:
import scipy.integrate as integrate

print integrate.trapz(x=pos, y=moment[:,0])/(pos[-1]-pos[0])

52008.17898737412
